In [1]:
#Basic Libraries
import numpy as np
import pandas as pd

#Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#Text Handling Libraries
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# clustering
from sklearn.cluster import KMeans

In [153]:
df_clean = pd.read_csv('clean.csv')
print(len(df_clean))
df_clean['product'] = df_clean['product'].drop_duplicates()
df_cleaned = df_clean.dropna()
print(len(df_cleaned))
df_cleaned.info()

18840
16217
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16217 entries, 0 to 18838
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        16217 non-null  int64  
 1   index             16217 non-null  int64  
 2   product           16217 non-null  object 
 3   category          16217 non-null  object 
 4   sub_category      16217 non-null  object 
 5   brand             16217 non-null  object 
 6   sale_price        16217 non-null  float64
 7   market_price      16217 non-null  float64
 8   type              16217 non-null  object 
 9   rating            16217 non-null  float64
 10  description       16217 non-null  object 
 11  category_new      16217 non-null  object 
 12  sub_category_new  16217 non-null  object 
 13  type_new          16217 non-null  object 
 14  soup              16217 non-null  object 
dtypes: float64(3), int64(2), object(10)
memory usage: 2.0+ MB


In [154]:
df_cleaned = df_cleaned.reset_index()
products = df_cleaned['product']
indices = pd.Series(df_cleaned.index, index=df_cleaned['product'])
df_cleaned[['soup','product']]

,soup,product
0,beauty hygiene hair care sri sri ayurveda ha...,Garlic Oil - Vegetarian Capsule 500 mg
1,kitchen garden pets storage accessories mast...,Water Bottle - Orange
2,cleaning household pooja needs trm lamp lamp...,"Brass Angle Deep - Plain, No.2"
3,cleaning household bins bathroom ware nakoda...,Cereal Flip Lid Container/Storage Jar - Assort...
4,beauty hygiene bath hand wash nivea bathing ...,Creme Soft Soap - For Hands & Body
...,...,...
16212,cleaning household mops brushes scrubs liao ...,Toilet Cleaning Brush - Round With Holder (Big)
16213,foodgrains oil masala organic staples organic...,Organic Powder - Garam Masala
16214,beauty hygiene fragrances deos layerr perfume,"Wottagirl! Perfume Spray - Heaven, Classic"
16215,gourmet world food snacks dry fruits nuts fab...,Peri-Peri Sweet Potato Chips


In [155]:
# recommendation
def get_tfid_recommendation(title, sim):
    idx = indices[title]
    sim_scores = list(enumerate(sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    product_indices = [i[0] for i in sim_scores]
    return products.iloc[product_indices]

In [156]:
tfidf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0,stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_cleaned['soup'])
tfid_cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [157]:
searchstr = 'Peri-Peri Sweet Potato Chips'
get_tfid_recommendation(searchstr,tfid_cosine_sim)

3399                      High Protein Soya Chips
7918                             Chia Seeds Chips
16215                Peri-Peri Sweet Potato Chips
7739                             Mixed Seed Chaat
15083                     Peri Peri Pumpkin Seeds
15644                  Roasted Soya Nuts - Wasabi
485                 Nacho Crisps - Cheese & Herbs
1010     On The Go - Peri Peri Nachos & Salsa Dip
1714                  Nacho Crisps - Tikka Masala
3348                      Nacho Crisps - Barbeque
Name: product, dtype: object

In [163]:
# recommendation
def get_recommendations(title,sim):
    idx = indices[title]
    #print(idx)
    sim_scores = list(enumerate(sim[idx]))
    #print(sim_scores)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:10]
    product_indices = [i[0] for i in sim_scores]
    return products.iloc[product_indices]

In [164]:
searchstr = 'Creme Soft Soap - For Hands & Body'
get_recommendations(searchstr,tfid_cosine_sim)

3174                   Creme Care Soap - For Hands & Body
7314                                    Soap - Creme Care
723                Lavender Try Me Personal Care Gift Set
2803           Cold Processed Skin Lavender Handmade Soap
3447    Cold Processed Anti Tan Tomato Handmade Soap F...
3462    Sea Cool Glycerin Soap - Cold Processed, 100% ...
4313    Cold Processed Apple Mangosteen Handmade Soap ...
4317                        Sandalwood Pure Glycerin Soap
6710    Jasmine Pure Glycerin Soap - Cold Processed, 1...
Name: product, dtype: object